In [ ]:
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

BEARER_TOKEN = os.getenv("BEARER_TOKEN")

if BEARER_TOKEN is None:
    raise ValueError("Bearer token not found. Check your .env file.")

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [ ]:
query = '"AI" OR "Machine Learning" OR "Data Science" -Politics -is:retweet lang:en'

response = client.search_recent_tweets(
    query=query,
    max_results=100
)

if response.data is None:
    raise ValueError("No tweets returned. Try a different query.")

tweets = [tweet.text for tweet in response.data]
df = pd.DataFrame(tweets, columns=["tweet"])


In [ ]:
analyzer = SentimentIntensityAnalyzer()

df["sentiment_score"] = df["tweet"].apply(
    lambda x: analyzer.polarity_scores(x)["compound"]
)

def label_sentiment(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

df["sentiment"] = df["sentiment_score"].apply(label_sentiment)

In [ ]:
df.to_csv("data/social_issues.csv", index=False)

print("✅ Sentiment analysis completed successfully")